# Play with pulp library and select squad with constraints using linear programming

In [1]:
import pandas as pd
import os

In [2]:
data_path = os.path.dirname(os.getcwd()) + '\\data\\raw\\Fantasy-Premier-League\\'

data_2021 = pd.read_csv(data_path + '2021-22/gws/merged_gw.csv')
data_2021.tail()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
18732,Kurt Zouma,DEF,West Ham,2.6,0,0,6,0,1.8,128,...,3,0.0,-1,10796,12551,1755,54,False,0,30
18733,Aaron Cresswell,DEF,West Ham,3.8,0,0,15,0,14.8,411,...,3,4.0,1,18750,35570,16820,54,False,0,30
18734,John Ruddy,GK,Wolves,0.5,0,0,0,0,0.0,452,...,2,0.0,0,63,170,107,43,True,0,30
18735,Wilfred Ndidi,MID,Leicester,0.5,0,0,0,0,0.0,216,...,2,0.0,0,-207,538,745,48,True,0,30
18736,Ryan Fredericks,DEF,West Ham,0.2,0,0,0,0,0.0,415,...,3,0.0,0,81,139,58,44,False,0,30


# Create solver on whole season data

In [3]:
# data_2021 grouped by name and sum of 'total_points'
gw_sum = data_2021[['name', 'team', 'position', 'total_points', 'value']]
gw_sum = gw_sum.groupby(['name', 'team', 'position']).agg({'value': 'max', 'total_points': 'sum'}).reset_index()
gw_sum['value'] = gw_sum['value'] / 10
gw_sum['position'] = gw_sum['position'].replace({'GK': 1, 'DEF': 2, 'MID': 3, 'FWD': 4})
gw_sum.sort_values(by='total_points', ascending=False).head(10)

,name,team,position,value,total_points
502,Mohamed Salah,Liverpool,3,13.3,222
683,Trent Alexander-Arnold,Liverpool,2,8.5,187
264,Heung-Min Son,Spurs,3,10.9,165
368,João Pedro Cavaco Cancelo,Man City,2,7.2,162
317,Jarrod Bowen,West Ham,3,7.1,153
695,Virgil van Dijk,Liverpool,2,6.8,150
40,Andrew Robertson,Liverpool,2,7.2,142
620,Sadio Mané,Liverpool,3,12.0,140
29,Alisson Ramses Becker,Liverpool,1,6.0,138
103,Bukayo Saka,Arsenal,3,6.7,138


In [4]:
from src.squad_selection.select_team import select_team

In [5]:
decisions, captain_decisions, sub_decisions = select_team(gw_sum.total_points.values, gw_sum.value.values, gw_sum.position.values, gw_sum.team.values)

d:\studies\informatyka\semestr_3\fpl_ai_manager\venv\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Total expected score = 1912.6


In [6]:
from src.squad_selection.select_team import print_selected_team

print_selected_team(gw_sum, decisions, captain_decisions, sub_decisions)

**Bukayo Saka** Points = 138, Price = 6.7

**Conor Coady** Points = 123, Price = 4.9

**Emmanuel Dennis** Points = 115, Price = 6.2

**Heung-Min Son** Points = 165, Price = 10.9

**Jarrod Bowen** Points = 153, Price = 7.1

...

**Ben Foster** Points = 60, Price = 4.1

**Joe Gelhardt** Points = 31, Price = 4.6

**Sam Greenwood** Points = 6, Price = 4.5


Captain:


**Mohamed Salah** Points = 222, Price = 13.3